In [16]:
from collections import deque

import gym
from gym import wrappers, logger

import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [4]:
env_name = 'CartPole-v1'

In [5]:
# def blotzmann(qvalues, tau=0.01):
#     prob = []
#     for qvalue in qvalues:
#         prob.append()
#     print("lol")

def epsilongreedy(qvalues, epsilon=0.5):
    _, max_ind = qvalues.max(0)
    return max_ind.item() if np.random.random() < epsilon else np.random.randint(len(qvalues))

In [29]:
class Net(nn.Module):
    def __init__(self, in_size, out_size):
        super(Net, self).__init__()
        print(in_size, out_size)
        self.in_l = nn.Linear(in_size, out_size)

    def forward(self, x):
        x = F.relu(self.in_l(x))
        return x

In [35]:
class Agent:
    def __init__(self, _net):
        self.mem = deque(maxlen=10000)
        self.net = _net

    def choose_action(self, _observation, _reward, _done):
        qvalues = self.qvalue(_observation)
        print(qvalues)
        action_ind = epsilongreedy(qvalues, epsilon=0.1)
        return action_ind
        # return self.action_space.sample()

    def experience(self, _state, _action, _reward, _next_state, _done):
        self.mem.append((_state, _action, _reward, _next_state, _done))

    def minibatch(self, size=5):
        batch_s = min(size, len(self.mem))
        return random.sample(self.mem, batch_s)

    def qvalue(self, state):
        inp = torch.tensor(state).type(torch.FloatTensor)
        # for w in self.W[:-1]:
        #     # inp = torch.cat((torch.Tensor([1.0]), inp))
        #     # inp = func(torch.mv(w, inp))
        #     inp = 1/(1 + torch.exp(-torch.mv(w, inp)))
        # return torch.mv(self.W[-1], inp.float())
        return self.net(inp)

In [36]:
# You can set the level to logger.DEBUG or logger.WARN if you
# want to change the amount of output.
logger.set_level(logger.INFO)

env = gym.make(env_name)

# You provide the directory to write to (can be an existing
# directory, including one with existing data -- all monitor files
# will be namespaced). You can also dump to a tempdir if you'd
# like: tempfile.mkdtemp().
outdir = '/tmp/random-agent-results'
env = wrappers.Monitor(env, directory=outdir, force=True, video_callable=False)
env.seed(0)
observation_size = env.observation_space.shape[0]
action_size = env.action_space.n

net = Net(observation_size, env.action_space.n)
agent = Agent(net)

episode_count = 1000
reward = 0
done = False

rewards = []

INFO: Making new env: CartPole-v1
INFO: Clearing 2 monitor files from previous run (because force=True was provided)
4 2


In [37]:
for i in range(episode_count):
    ob = env.reset()
    rewardSum = 0
    steps = 0
    while not done:
        action = agent.choose_action(ob, reward, done)
        current_state = ob
        ob, reward, done, _ = env.step(action)
        rewardSum += reward
        steps += 1

        # Experience replay
        agent.experience(current_state, action, reward, ob, done)
        minibatch = agent.minibatch(size=10)
        if done:
            rewards.append(rewardSum)
            break



env.close()

tensor([0.1720, 0.3718], grad_fn=<ReluBackward0>)
tensor([0.3122, 0.2160], grad_fn=<ReluBackward0>)
tensor([0.1757, 0.3670], grad_fn=<ReluBackward0>)
tensor([0.0376, 0.5185], grad_fn=<ReluBackward0>)
tensor([0.0000, 0.6713], grad_fn=<ReluBackward0>)
tensor([0.0329, 0.5183], grad_fn=<ReluBackward0>)
tensor([0.0000, 0.6738], grad_fn=<ReluBackward0>)
tensor([0.0257, 0.5234], grad_fn=<ReluBackward0>)
tensor([0.1614, 0.3734], grad_fn=<ReluBackward0>)
tensor([0.0202, 0.5311], grad_fn=<ReluBackward0>)
tensor([0.1554, 0.3821], grad_fn=<ReluBackward0>)
tensor([0.0136, 0.5407], grad_fn=<ReluBackward0>)
tensor([0.1482, 0.3927], grad_fn=<ReluBackward0>)
tensor([0.2845, 0.2443], grad_fn=<ReluBackward0>)
tensor([0.4228, 0.0949], grad_fn=<ReluBackward0>)
tensor([0.2850, 0.2513], grad_fn=<ReluBackward0>)
tensor([0.1457, 0.4075], grad_fn=<ReluBackward0>)
tensor([0.0045, 0.5644], grad_fn=<ReluBackward0>)
tensor([0.1396, 0.4148], grad_fn=<ReluBackward0>)
tensor([0.2764, 0.2650], grad_fn=<ReluBackward0>)
